# Assignment #7: The space of rotations SO(3) and the numerical integration of rigid body equations. 

* Save this file in your Institutional G-Suite google drive and rename it starting with your index number. For example if you are E/90/512 then rename the file to 
'E90512_xxxx.ipynb'

* Get started!

* Once complete share with smaithri@eng.pdn.ac.lk and chanakatb@eng.pdn.ac.lk

* Submit the link at the end of 'Lesson on Rigid Body Motion'

* Upload the file through the Assignment Submission Link at the end of the Lesson.

# Setting up Python 

In [1]:
import math
import numpy as np
from numpy import linalg
from scipy.integrate import odeint
import plotly.graph_objects as go

from sympy import symbols
from sympy import *

from sympy.physics.mechanics import dynamicsymbols, init_vprinting

## Python Rigidbody Class Definition

In [11]:
class mugas_rigid_body_functions:
  
  def __init__(self): 
    self=[];

  def simulateDySystem(self, dynamicSystemModel, Tmax, dT, X0, sysPara, figTitle, xLabel, yLabel):
    t = np.linspace(0, Tmax, int(Tmax/dT+1))
    sol = odeint(dynamicSystemModel, X0, t, args=(sysPara,));
    self.soln=sol;
    fig = go.Figure()
    [nt,nS]=sol.shape;

    # Add traces
    for nn in range(nS):
      fig.add_trace(go.Scatter(x=t, y=sol[:,nn], mode='lines+markers', name='x'+str(nn+1)))
      fig.update_layout(title=figTitle, xaxis=dict(title=xLabel),yaxis=dict(title=yLabel))
    
    fig.show()
    return [t,sol,fig] 

  def hatMatrix(self, X):
    return np.array([[0., -X[2], X[1]],[X[2], 0., -X[0]],[-X[1], X[0], 0.]])

  def qFromAxisAngles(self, theta, unitAxis):
    return np.concatenate(([np.cos(theta/2)],np.sin(theta/2)*np.array(unitAxis)))

  def RfromQuaternions(self,q):
    q0=q[0]; w=q[1:];
    R=np.identity(3)+2*q0*self.hatMatrix(w)+2*self.hatMatrix(w) @ self.hatMatrix(w);
    return R

  def rotationMatrix2EulerAngles(self,R):
    if R[2,2] <1:
      if R[2,2] > -1:
        theta2=math.acos(R[2,2]);
        theta1=math.atan2(R[0,2],R[1,2]);
        theta3=math.atan2(R[2,0],-R[2,1]);
      else: #R[2,2]=-1 No Unique solution 
        theta2=np.pi;
        theta1=-math.atan2(R[0,1],-R[0,0]);
        theta3=0;
    else: # R[2,2] = +1 No Unique solution
      theta2 = 0;
      theta1 = math.atan2(R[0,1],R[0,0]) ;
      theta3 = 0;
    return [np.pi-theta1,theta2,np.pi-theta3]

  def Re3equalsgamma(self,gamma):
    theta=math.acos(gamma[2]);
    n1=-gamma[1]/np.sin(theta);
    n2=gamma[0]/np.sin(theta);
    q00=self.qFromAxisAngles(theta,[n1,n2,0]);

    return self.RfromQuaternions(q00)

  def rotate_and_translate(self,objectVertices,R,b):
    #object vertices should be given as an numpy array of shape (3,n) 
    #Ex - X=[0, 0, 2, 2, 0, 0, 2, 2], Y=[0, 2, 2, 0, 0, 2, 2, 0], Z=[0, 0, 0, 0, 1, 1, 1, 1]; objectVertices=[X,Y,Z]
    return np.array([[b[0],b[1],b[2]]]).T + R @ objectVertices



  def addOrthNormFrame(self, fig, o, R, axisRange, axisColor):
    e1=[1,0,0]; e2=[0,1,0]; e3=[0,0,1];
    b1=R @ e1; b2=R @ e2; b3=R @ e3;
    fig.add_trace(go.Scatter3d(x=[str(o[0]),str(o[0]+b1[0])], y=[str(o[1]),str(o[1]+b1[1])], z=[str(o[2]),str(o[2]+b1[2])], hoverinfo='x+y+z', mode='lines', line=dict(width=8, color=axisColor)))
    fig.add_trace(go.Scatter3d(x=[str(o[0]),str(o[0]+b2[0])], y=[str(o[1]),str(o[1]+b2[1])], z=[str(o[2]),str(o[2]+b2[2])], hoverinfo='x+y+z', mode='lines', line=dict(width=8, color=axisColor)))
    fig.add_trace(go.Scatter3d(x=[str(o[0]),str(o[0]+b3[0])], y=[str(o[1]),str(o[1]+b3[1])], z=[str(o[2]),str(o[2]+b3[2])], hoverinfo='x+y+z', mode='lines', line=dict(width=8, color=axisColor)))
    fig.update_layout(showlegend=False, scene=dict(xaxis=dict(range=axisRange[0], autorange=False), yaxis=dict(range=axisRange[1], autorange=False), zaxis=dict(range=axisRange[2], autorange=False), aspectratio=dict(x=1, y=1, z=1)))
    return fig

  def cube_vertices(self,cubeDimensions):
    l=cubeDimensions[0]; w=cubeDimensions[1]; h=cubeDimensions[2];
    X=[0, 0, l, l, 0, 0, l, l]; Y=[0, w, w, 0, 0, w, w, 0]; Z=[0, 0, 0, 0, h, h, h, h]; XX=[X,Y,Z];
    #X=[-l, -l, l, l, -l, -l, l, l]; Y=[-w, w, w, -w, -w, w, w, -w]; Z=[-h, -h, -h, -h, h, h, h, h]; XX=[X,Y,Z]; 
    return XX


  def animate_particle_motion(self, XX, axisRange, figTitle):
    xx=[jj[0] for jj in XX]
    yy=[jj[1] for jj in XX]
    zz=[jj[2] for jj in XX]

    fig = go.Figure(
        data=[go.Scatter3d(x=[xx[0]], y=[yy[0]],z=[zz[0]],
                        mode="markers",
                        marker=dict(color="red", size=10)),
              go.Scatter3d(x=xx, y=yy,z=zz, name='Path',
                        mode="lines",
                        line=dict(color="blue", width=2))],
        layout=go.Layout(
            scene = dict(
                        xaxis=dict(range=axisRange[0], autorange=False),
                        yaxis=dict(range=axisRange[1], autorange=False),
                        zaxis=dict(range=axisRange[2], autorange=False),
                        aspectratio=dict(x=1, y=1, z=1),
            ),
            title_text=figTitle, hovermode="closest",
            updatemenus=[dict(type="buttons",
                              buttons=[dict(label="Play",
                                            method="animate",
                                            args=[None])])]),
        frames=[go.Frame(
            data=[go.Scatter3d(
                x=[p[0]],
                y=[p[1]],
                z=[p[2]],
                name='Particle',
                mode="markers",
                marker=dict(color="red", size=10))])

            for p in XX]
    )

    fig.show()
    return fig

  def animated_cube_flat_shading(self, cubeVertices,figTitle):
    fig = go.Figure(
        frames=[go.Frame(data=[
          go.Mesh3d(
            # 8 vertices of a cube
            x=xx[0][0],
            y=xx[0][1],
            z=xx[0][2],
            # i, j and k give the vertices of triangles
            i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
            j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
            k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
            name='y',
            opacity=0.6,
            color='#DC143C',
            flatshading = True)]) for xx in cubeVertices])

    fig.add_trace(go.Mesh3d(
            # 8 vertices of a cube
            x=cubeVertices[0][0][0],
            y=cubeVertices[0][0][1],
            z=cubeVertices[0][0][2],
            # i, j and k give the vertices of triangles
            i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
            j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
            k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
            name='y',
            opacity=0.6,
            color='#DC143C',
            flatshading = True)
        )

    fig.update_layout(
          title=figTitle,
          width=600,
          height=600,
          scene=dict(
                      xaxis=dict(range=[-5., 5.], autorange=False),
                      yaxis=dict(range=[-5., 5.], autorange=False),
                      zaxis=dict(range=[-5., 5.], autorange=False),
                      aspectratio=dict(x=1, y=1, z=1),
                      ),
          updatemenus=[dict(type="buttons",
                              buttons=[dict(label="Play",
                                            method="animate",
                                            args=[None])])])
    len(fig.frames)
    fig.show()
    return fig
  
  def simulating_a_cube(self, dt, Tmax, cubeDimensions, parameters,ICs):
    XX=self.cube_vertices(cubeDimensions);
    
    #Xs=self.eulers_method(dt,Tmax,parameters,ICs);
    Xs=self.runga_kutta_method(dt,Tmax,parameters,ICs);
    ICR=ICs[0][0];
    XX0=ICR @ XX;
    
    rotatedVertices=[[XX0]]
    for X in Xs:
      #print(X[0])
      R=X[0][0]; 
      o=X[0][1];
      XXi=self.rotate_and_translate(XX,R,o);
      XX0=XXi;
      rotatedVertices+=[[XX0]];
    return rotatedVertices


  def eulers_method(self, dt,Tmax,parameters,ICs):
    M=parameters['M']; II=parameters['II'];
    invII=linalg.inv(II) ;
    timeSteps=np.arange(0,Tmax+dt,dt);
    R=ICs[0][0]; o=ICs[0][1];
    omega=ICs[1]; doto=ICs[2]; Xc=ICs[3];
    X=ICs;
    spi=R@II@R.T@omega; p=M*doto;
    Xout=[X];
    for t in timeSteps:
      [taue,fe]=externalForceModel(self, parameters,X);
      [taua,fa]=actuator(self, parameters, t, X,taue,fe);
      
      dspi=taue+taua;
      dp=fe+fa;
      #dXc=controller_dynamics(self, t,X,taue,fe,parameters)
      
      if linalg.norm(omega)>=0.0001:
        nomega=omega/linalg.norm(omega);
        thetaomegat=dt*linalg.norm(omega);
      else:
        nomega=np.array([0,0,0]);
        thetaomegat=0;
      qomegat=np.concatenate(([np.cos(thetaomegat/2)],np.sin(thetaomegat/2)*nomega));
      R=self.RfromQuaternions(qomegat) @ R;
      o=o+dt*doto;
      spi=spi+dt*dspi;
      p=p+dt*dp;
      doto=p/M;
      omega=R @ invII @ R.T @ spi;
      Xc=Xc#+dt*dXc;
      X=[[R,o],omega,doto, Xc];
      Xout+=[X]
        
    return Xout

  def runga_kutta_method(self, dt,Tmax,parameters,ICs):
    M=parameters['M']; II=parameters['II'];
    invII=linalg.inv(II) ;
    timeSteps=np.arange(0,Tmax+dt,dt);
    R=ICs[0][0]; o=ICs[0][1];
    omega=ICs[1]; doto=ICs[2]; Xc=ICs[3];
    X=ICs;
    spi=R@II@R.T@omega; p=M*doto;
    Xout=[X];
    for t in timeSteps:
      Y1=self.rk4_function(0.5*dt, X, t, X, parameters);
      Y2=self.rk4_function(0.5*dt, X, t+0.5*dt, Y1, parameters);
      Y3=self.rk4_function(dt, X, t+0.5*dt, Y2, parameters);

      [thetaomega1,nomega1,doto1,dp1,dspi1,dXc1]=self.rigid_body_system(parameters, t, X);
      [thetaomega2,nomega2,doto2,dp2,dspi2,dXc2]=self.rigid_body_system(parameters, t+0.5*dt, Y1);
      [thetaomega3,nomega3,doto3,dp3,dspi3,dXc3]=self.rigid_body_system(parameters, t+0.5*dt, Y2);
      [thetaomega4,nomega4,doto4,dp4,dspi4,dXc4]=self.rigid_body_system(parameters, t+dt, Y3);
      omegak=(dt/6.0)*(thetaomega1*nomega1+2*thetaomega2*nomega2+2*thetaomega3*nomega3+thetaomega4*nomega4)
      if linalg.norm(omegak)>=0.0001:
        nomegak=omegak/linalg.norm(omegak);
        thetaomegak=linalg.norm(omegak);
      else:
        nomegak=np.array([0,0,0]);
        thetaomegak=0;
      qomegak=np.concatenate(([np.cos(thetaomegak/2)],np.sin(thetaomegak/2)*nomegak));
      Rk=self.RfromQuaternions(qomegak) @ X[0][0];
      
      dotoko=(dt/6.0)*(doto1+2*doto2+2*doto3+doto4);
      dpk=(dt/6.0)*(dp1+2*dp2+2*dp3+dp4);
      dspik=(dt/6.0)*(dspi1+2*dspi2+2*dspi3+dspi4);
      dXck=(dt/6.0)*(dXc1+2*dXc2+2*dXc3+dXc4);
      
      ok=X[0][1]+dotoko;
      spik=X[0][0]@II@X[0][0].T@X[1]+dspik;
      dotok=X[2]+dpk/M;
      Xck=X[3]+dXck;
      omegak=Rk @ linalg.inv(II) @ Rk.T @ spik;
        
      X=[[Rk,ok],omegak,dotok,Xck];
      Xout+=[X]   
    return Xout

  def rk4_function(self, dtk, X, tk, Xk, parameters):
    M=parameters['M']; II=parameters['II'];
    [thetaomega1,nomega1,doto1,dp1,dspi1,dXc1]=self.rigid_body_system(parameters, tk, Xk)
    qomega1=np.concatenate(([np.cos(dtk*thetaomega1/2)],np.sin(dtk*thetaomega1/2)*nomega1));
    R1=self.RfromQuaternions(qomega1)@X[0][0];
    spi1=X[0][0]@II@X[0][0].T@X[1]+dtk*dspi1; omega1=R1@linalg.inv(parameters['II'])@R1.T@spi1;
    X1=[[R1, X[0][1]+dtk*doto1],omega1,X[2]+dtk*dp1/M, X[3]+dtk*dXc1]
    return X1

  def rigid_body_system(self, parameters, t, X):
    [taue,fe]=externalForceModel(self, parameters,X);
    [taua,fa]=actuator(self, parameters, t, X,taue,fe);
    dspi=taue+taua;
    dp=fe+fa;
    dXc=np.array([0,0,0]); #controller_dynamics(self,t,X,taue,fe,parameters)
    omega=X[1]; doto=X[2]
    if linalg.norm(omega)>=0.0001:
      nomega=omega/linalg.norm(omega);
      thetaomega=linalg.norm(omega);
    else:
      nomega=np.array([0,0,0]);
      thetaomega=0;
    return [thetaomega,nomega,doto,dp,dspi,dXc] 

In [ ]:
def LinearSystemModel(X, t, A):
  dXdt=A @ X;
  return dXdt

In [12]:
mr=mugas_rigid_body_functions()

In the section 'Infinitesimal Rotations' we saw that $R \in  SO(3)$ can be thought of as a map that transforms points in space to other points in space in such a way that it preserves distances between points and angles between lines. That is, every $R\in \mathrm{SO}(3)$ can be viewed as a rigid rotation of space. Conversely, since we have seen that two frames with coinciding origins are related by $\mathbf{b}=\mathbf{e}\,R$, every rigid body rotation about a fixed point can also be identified with an $R\in \mathrm{SO}(3)$ by fixing an orthonormal frame $\mathbf{b}$ on the rigid body and finding its relationship with some inertial frame-$\mathbf{e}$. We also saw that $R\in \mathrm{SO}(3)$ can be seen as a coodinate transformation between the two frames $\mathbf{b}$ and $\mathbf{e}$.

In what follows we will take a closer look at all possible rigid body rotations. Or in other words we will try to get some insight about the structure of the space $\mathrm{SO}(3)$.



# Q#1 Prove Euler's Theorem

* Show that the eignevalues of any $R\in SO(3)$ are of the 
form $\{1,e^{i\theta},e^{-i\theta}\}$. 

* Thus show that for any $R\in SO(3)$ there exists a $V\in \mathbb{R}^3$ such that $RV=V$.  

* Shows that there exists transform, $T$, of both the frames $\mathbf{e}$ and $\mathbf{b}$ such that with respect to these transformed frames $R$ takes the form. 

\begin{align}
\begin{bmatrix}
1 & 0 & 0\\
0 & \cos{\theta} & -\sin{\theta}\\
0 & \sin{\theta} & \cos{\theta}
\end{bmatrix}.
\end{align} 

# Q#2 


Show that the space of rotations, $\mathrm{SO}(3)$ can be identified with the solid ball in $\mathbb{R}^3$ with the antipodal points on the boundary identified or by the space of unit tangent vectors on $\mathbb{S}^2$ that is typically denoted by $T_0\mathbb{S}^2$. 

# Q#3 Euler Angle Parameterization of $SO(3)$

Write down a python method for symbolically and numericall computing the $i-j-k$ Euler angles.


#Q#4 Unit Quaternion Parameterization of $SO(3)$

Consider the case where an orthonormal frame $\mathbf{b}(t)$ is moving with respect to another orthonormal frame $\mathbf{e}$ such that the origins of the two frames remain coincident. Let $\mathbf{b}(t)=\mathbf{e}\,R(t)$ and $\widehat{\Omega}=R^T\dot{R}$. Consider $\Omega$ and $\omega=R\Omega$ (note that $\widehat{\omega}=\dot{R}R^T$).



## Q#4.1 

If $\Omega$ is a constant, show that, the solution to the following initial value problem 
\begin{align}
\dot{R}&=R\widehat{\Omega},\:\:\:\:\:\:\: R(0)=I_{3\times 3},
\end{align}
is explicitly and uniquely given by
\begin{align}
{R}(t)&=\exp{(t\widehat{\Omega})},
\end{align}
where $\exp{(A)}$ denotes the matrix exponential of the matrix $A$. 

## Q#4.2 

If $\omega$ is a constant, show that, the solution to the following initial value problem 
\begin{align}
\dot{R}&=\widehat{\omega}R,\:\:\:\:\:\:\: R(0)=I_{3\times 3},
\end{align}
is explicitly and uniquely given by
\begin{align}
{R}(t)&=\exp{(t\widehat{\omega})},
\end{align}
where $\exp{(A)}$ denotes the matrix exponential of the matrix $A$. 

## Q#4.3 

Show that for any given $R$ there exists some $\widehat{\Omega}\in \mathrm{so}(3)$ such that $R=\exp{(\widehat{\Omega})}$. Thus we see that the map $\exp : \mathrm{so}(3) \to \mathrm{SO}(3)$ is an onto map.

## Q#4.4

Show that every $R\in SO(3)$ can be written down as
\begin{align}
R=\exp{\left(\theta\,{\widehat{n}}\right)}&=I+\sin{\theta}\,\widehat{n}+(1-\cos{\theta})\,\widehat{n}^2.
\end{align}


##Q#4.5

Show that every $R \in \mathrm{SO}(3)$ can be written down as 
\begin{align*}
R&=I+2q_0\widehat{w}+2\widehat{w}^2.
\end{align*}
for some $(q_0,w)\in\mathbb{S}^3$. 


## Q#4.6

Show that the map $\mathbb{S}^3 \mapsto SO(3)$ that is defined above is a 2:1 map.

## Q#4.7

Show that the unit quaternion $q=(q_0,w)$ that correspond to a given rotation matrix $R$ can be found using the following two expressions:
\begin{align}
\mathrm{trace}(R)&=-1+4q_0^2=2\cos \theta +1,\\
R-R^T&=4q_0 \widehat{w}=4\cos{\left(\frac{\theta}{2}\right)} \widehat{w}.
\end{align}
The first expression determines $q_0=\cos{\frac{\theta}{2}}$ and the second expression determines $w$.

##Q#4.8 Develop a Python method to find the axis and angle of rotation of a given $R\in SO(3)$

In [4]:
R11, R12, R13, R21, R22, R23, R31, R32, R33, q_0, w_1, w_2, w_3, theta_1, theta_2, theta_3, alpha, beta, gamma=symbols('R11, R12, R13, R21, R22, R23, R31, R32, R33, q_0, w_1, w_2, w_3, theta_1, theta_2, theta_3, alpha, beta, gamma')

In [5]:
R=Matrix([[R11, R12, R13], [R21, R22, R23], [R31, R32, R33]])
R1=Matrix([[1,0,0],[0, cos(theta_1), -sin(theta_1)], [0, sin(theta_1), cos(theta_1)]]);
R2=Matrix([[cos(theta_2),0, sin(theta_2)], [0,1,0], [-sin(theta_1),0, cos(theta_1)]]);
R3=Matrix([[cos(theta_3), -sin(theta_3), 0], [sin(theta_3), cos(theta_3),0],[0,0,1]]);

In [6]:
R313=R3.subs(theta_3,alpha)@R1.subs(theta_1,beta)@R3.subs(theta_3,gamma)

In [ ]:
R313

Matrix([
[-sin(alpha)*sin(gamma)*cos(beta) + cos(alpha)*cos(gamma), -sin(alpha)*cos(beta)*cos(gamma) - sin(gamma)*cos(alpha),  sin(alpha)*sin(beta)],
[ sin(alpha)*cos(gamma) + sin(gamma)*cos(alpha)*cos(beta), -sin(alpha)*sin(gamma) + cos(alpha)*cos(beta)*cos(gamma), -sin(beta)*cos(alpha)],
[                                    sin(beta)*sin(gamma),                                     sin(beta)*cos(gamma),             cos(beta)]])

In [7]:
q0=simplify(sqrt((trace(R313)+1)/4))

In [ ]:
print(q0)

sqrt(cos(beta)*cos(alpha + gamma) + cos(beta) + cos(alpha + gamma) + 1)/2


In [ ]:
whatTemp=simplify((R313-R313.T)/(4*q0))

In [ ]:
wTemp=Matrix([-whatTemp[1,2],whatTemp[0,2],-whatTemp[0,1]])

In [ ]:
print(wTemp[2])

(cos(beta) + 1)*sin(alpha + gamma)/(2*sqrt(cos(beta)*cos(alpha + gamma) + cos(beta) + cos(alpha + gamma) + 1))


In [ ]:
0.57337881971989**2+(-0.51286648792615)**2+0.17052789820977**2+0.61573121619021**2

In [ ]:
[*simplify(R313.eigenvals())][2]

sqrt(cos(beta)**2*cos(alpha + gamma)**2 + 2*cos(beta)**2*cos(alpha + gamma) + cos(beta)**2 + 2*cos(beta)*cos(alpha + gamma)**2 - 2*cos(beta) + cos(alpha + gamma)**2 - 2*cos(alpha + gamma) - 3)/2 + cos(beta)*cos(alpha + gamma)/2 + cos(beta)/2 + cos(alpha + gamma)/2 - 1/2

In [ ]:
print([*simplify(R313.eigenvals())][2])

sqrt(cos(beta)**2*cos(alpha + gamma)**2 + 2*cos(beta)**2*cos(alpha + gamma) + cos(beta)**2 + 2*cos(beta)*cos(alpha + gamma)**2 - 2*cos(beta) + cos(alpha + gamma)**2 - 2*cos(alpha + gamma) - 3)/2 + cos(beta)*cos(alpha + gamma)/2 + cos(beta)/2 + cos(alpha + gamma)/2 - 1/2


In [ ]:
tanTheta=((sqrt(-(cos(beta)**2*cos(alpha + gamma)**2 + 2*cos(beta)**2*cos(alpha + gamma) + cos(beta)**2 + 2*cos(beta)*cos(alpha + gamma)**2 - 2*cos(beta) + cos(alpha + gamma)**2 - 2*cos(alpha + gamma) - 3))/2)/(cos(beta)*cos(alpha + gamma)/2 + cos(beta)/2 + cos(alpha + gamma)/2 - 1/2))

In [ ]:
wHat=Matrix([[0,-w_3,w_2],[w_3,0,-w_1],[-w_2,w_1,0]])

In [ ]:
Rq=eye(3)+2*q_0*wHat+2*wHat@wHat

In [ ]:
Rq

Matrix([
[-2*w_2**2 - 2*w_3**2 + 1,   -2*q_0*w_3 + 2*w_1*w_2,    2*q_0*w_2 + 2*w_1*w_3],
[   2*q_0*w_3 + 2*w_1*w_2, -2*w_1**2 - 2*w_3**2 + 1,   -2*q_0*w_1 + 2*w_2*w_3],
[  -2*q_0*w_2 + 2*w_1*w_3,    2*q_0*w_1 + 2*w_2*w_3, -2*w_1**2 - 2*w_2**2 + 1]])

In [ ]:
1/np.sqrt(2)

0.7071067811865475

#Q#5 Integration of Rotational Kinematics

Consider a discrete sequence of time points $\{t_0,t_1,\cdots, t_k,t_{k+1},\cdots,t_N\}$ at which we need to aproximate the solution of our initial value problem.

\begin{align}
\dot{R}=R\widehat{\Omega},\:\:\:\: R(t_0)=R_0
\end{align}

 Let $R_k\triangleq R(t_k)$ and $\Omega_k\triangleq \Omega(t_k)$ and $\Delta t_k\triangleq (t_{k+1}-t_k)$. The assuming that $\Omega(t)\approx \Omega(t_k)$  during the time interval $[t_k,t_{k+1}]$ we see that

\begin{align}
R_{k+1}&\approx R_k\exp{(\Delta t_k\Omega_k)}
\end{align}
with $R_0=R(0)$. The accuracy of this approximation increases as $\Delta t_k$ beocmes small.

Note that we can explicitly compute $\exp{(\Delta t_k\Omega_k)}$ using the Euler-Rodrigues formula 
\begin{align}
\exp{(\Delta t_k\Omega_k)}=I+2{q_0}_k\widehat{w}_k+2\widehat{w}^2_k
\end{align}
where 
\begin{align}
\theta_k&=\Delta t_k||\Omega_k||,\\
{q_0}_k&=\cos{\left(\frac{\theta_k}{2}\right)},\\
w_k &= \sin{\left(\frac{\theta_k}{2}\right)}\frac{\Omega_k}{||\Omega_k||}.
\end{align}

**Note that the same scheme for the initial value problem $\dot{R}=\widehat{\omega}R$, with $R(t_0)=R_0$**

## Q#5.1

Develop a python method to numerically compute the solution to the initial value problem 
\begin{align}
\dot{R}=R\widehat{\Omega},\:\:\:\: R(t_0)=R_0
\end{align}
for a given time varying $\Omega(t)$

##Q#5.2 

If the time varying orthonomal frame $\mathbf{b}(t)$ is such that $\mathbf{b}(t)=\mathbf{e}R(t)$ where $R(t)$ satisfies the above initial condition problem develop a python method to animate the motion of $\mathbf{b}(t)$ for a given time varying $\Omega(t)$.

# Q#6 Integrating Rigid Body Equations

## Q#6.1 Based on the Eulers scheme develop a Python method to numerically integrate the rigid body dynamics given by 

\begin{align}
\dot{\bar{x}}'&=\omega \times \bar{x}'\\
\dot{R}&=\widehat{\omega}R,\\
\dot{p}&=f^e,\\
\dot{\pi}&=\omega \times \bar{x}'\times p+\tau_e
\end{align} 

##Q#6.2 **Extra Credit** - 4th Order Runga-Kutta numerical scheme for Integrating Rigid Body Motion using the spatial frame equations

Consider the dynamic system $\dot{x}=F(t,x)$ defined on $\mathbb{R}^n$ then the 4th order Runga-Kutta numerical integrations scheme for this system is given by
\begin{align}
k_1&=F(t_k\,,x_k),\\
k_2&=F(t_k+0.5h \, ,x_k+ 0.5h\,k_1),\\
k_3&=F(t_k+0.5h \, ,x_k+ 0.5h\,k_2),\\
k_4&=F(t_k+h\, ,x_k+ h\,k_3),\\
x_{k+1}&=x_k+\frac{h}{6}\left(k_1+2k_2+2k_3+k_4
  \right)
\end{align}

For rigid body mechanical systems described in the spatial frame the equations of motion are given by
\begin{align}
\dot{\bar{x}}'&=\omega \times \bar{x}'\\
\dot{R}&=\widehat{\omega}R,\\
\dot{p}&=f^e,\\
\dot{\pi}&=\omega \times \bar{x}'\times p+\tau_e
\end{align} 

When converted to a dynamic system form we have $\dot{X}=F(t,X)$ where
\begin{align}
X(t)&\triangleq \begin{bmatrix}R(t) & o(t) & \omega(t) & p(t)
\end{bmatrix}^T\\
F(t,X(t))&\triangleq \begin{bmatrix}\omega(t) & \frac{1}{M}p(t) & \tau(t,X) & f(t,X)
\end{bmatrix}^T
\end{align}


### Q6.2.1 Show that the following numerical scheme is an extension of the 4th order Runga-Kutta method for mechanical systems



\begin{align}
X(t_k)&\triangleq \begin{bmatrix}R(t_k) & o(t_k) & \omega(t_k) & p(t_k)
\end{bmatrix}^T\\
F(t_k,X(t_k))&\triangleq \begin{bmatrix}\omega(t_k) & \frac{1}{M}p(t_k) & \tau(t_k,X_k) & f(t_k,X_k)
\end{bmatrix}^T
\end{align}

Let 

---

\begin{align}
R_1(t_k+0.5h)&=\exp{\left(0.5h\widehat{\omega}(t_k)\right)}\,R(t_k)\\
o_1(t_k+0.5h)&=o(t_k)+\frac{0.5h}{M}p(t_k)\\
\omega_1(t_k+0.5h)&=R_1(t_k+0.5h)\mathbb{I}^{-1}R_1^T(t_k+0.5h)\left(R(t_k)\mathbb{I}R(t_k)^T\omega(t_k)+ 0.5h\,\tau(t_k,X(t_k))\right)\\
p_1(t_k+0.5h)&=p(t_k)+0.5h\,f(t_k,X(t_k))
\end{align}
and
\begin{align}
X_{1_k}&\triangleq \begin{bmatrix}R_1(t_k+0.5h) & o_1(t_k+0.5h) & \omega_1(t_k+0.5h) & p_1(t_k+0.5h)
\end{bmatrix}
\end{align}

---

\begin{align}
R_2(t_k+0.5h)&=\exp{\left(0.5h\widehat{\omega}_1(t_k+0.5h)\right)}\,R(t_k)\\
o_2(t_k+0.5h)&=o(t_k)+\frac{0.5h}{M}p_1(t_k+0.5h)\\
\omega_2(t_k+0.5h)&=R_2(t_k+0.5h)\mathbb{I}^{-1}R_2^T(t_k+0.5h)\left(R(t_k)\mathbb{I}R(t_k)^T\omega(t_k)+ 0.5h\,\tau(t_k+0.5h,X_1(t_k+0.5h))\right)\\
p_2(t_k+0.5h)&=p(t_k)+0.5h\,f(t_k+0.5h,X_1(t_k+0.5h))
\end{align}
and
\begin{align}
X_{2_k}&\triangleq \begin{bmatrix}R_2(t_k+0.5h) & o_2(t_k+0.5h) & \omega_2(t_k+0.5h) & p_2(t_k+0.5h)
\end{bmatrix}
\end{align}

---

\begin{align}
R_3(t_k+h)&=\exp{\left(h\widehat{\omega}_2(t_k+h)\right)}\,R(t_k)\\
o_3(t_k+h)&=o(t_k)+\frac{h}{M}p_2(t_k+0.5h)\\
\omega_3(t_k+h)&=R_3(t_k+h)\mathbb{I}^{-1}R_3^T(t_k+h)\left(R(t_k)\mathbb{I}R(t_k)^T\omega(t_k)+ h\,\tau(t_k+h,X_2(t_k+0.5h))\right)\\
p_3(t_k+h)&=p(t_k)+h\,f(t_k+h,X_2(t_k+0.5h))
\end{align}
and
\begin{align}
X_{3_k}&\triangleq \begin{bmatrix}R_3(t_k+h) & o_3(t_k+h) & \omega_3(t_k+h) & p_3(t_k+h)
\end{bmatrix}
\end{align}

---

Thus we have
\begin{align}
R(t_k+h)&=\exp{\left(h\widehat{\omega}_k\right)}\,R(t_k)\\
o(t_k+h)&=o(t_k)+\frac{h}{M}p_k\\
\omega(t_k+h)&=R(t_k+h)\mathbb{I}^{-1}R^T(t_k+h)\left(R(t_k)\mathbb{I}R(t_k)^T\omega(t_k)+ h\,\tau_k\right)\\
p(t_k+h)&=p(t_k)+h\,f_k
\end{align}
and
\begin{align}
X(t_k+h)&\triangleq \begin{bmatrix}R(t_k+h) & o(t_k+h) & \omega(t_k+h) & p(t_k+h)
\end{bmatrix}
\end{align}
where
\begin{align}
\tau_k&=\frac{1}{6}\left(\tau(t_k,X(t_k))+2\tau(t_k+0.5h,X_1(t_k+0.5h))+2\tau(t_k+0.5h,X_2(t_k+0.5h))+\tau(t_k+h,X_3(t_k+h))\right)\\
\omega_k&=\frac{1}{6}\left(\omega(t_k)+2\omega_1(t_k+0.5h)+2\omega_2(t_k+0.5h)+\omega_3(t_k+h)
\right)\\
f_k&=\frac{1}{6}\left(f(t_k,X(t_k))+2f(t_k+0.5h,X_1(t_k+0.5h))+2f(t_k+0.5h,X_2(t_k+0.5h))+f(t_k+h,X_3(t_k+h))\right)\\
p_k&=\frac{1}{6}\left(p(t_k)+2p_1(t_k+0.5h)+2p_2(t_k+0.5h)+p_3(t_k+h)
\right)
\end{align}

### Q6.2.2 Develop a Python method for integrating rigid body equations using the above scheme

#### Numerical Example

Consider $\dot{R}=R\widehat{\Omega}$ where $\Omega(t)=[2\sin{t},2\cos{t},0]$. We are intersted in finding the rotation matrix $R$ when $t=2\pi$ with $R(0)=I_{3\times 3}$.

Consder the equally spaced sequence of time points $[0,t_1,t_2,\cdots,2\pi]$ where $\Delta t_k=2\pi/10$.

In [31]:
h, t_0, t_1, t_2=symbols('h, t_0, t_1, t_2')

In [36]:
R0=np.eye(3);
listR=[R0];
t=[t_0,t_1,t_2];
for k in t:
  Omega_k=2*[sin(k),cos(k),0];
  theta_k=h*2;
  n_k=[Omega_k[0]/2,Omega_k[1]/2,Omega_k[2]/2];
  hatn_k=Matrix([[0,-n_k[2],n_k[1]],[n_k[2],0,-n_k[0]],[-n_k[1],n_k[0],0]])
  R0=R0 @ (eye(3)+sin(theta_k)*hatn_k+(1-cos(theta_k))*hatn_k @ hatn_k);
  listR+=[R0]

In [37]:
print(simplify(trace(listR[2])))

-0.25*sin(h)**4*sin(t_0)**2 + 0.5*sin(h)**4*sin(t_0)*sin(t_1)*cos(t_0 - t_1) - 0.25*sin(h)**4*sin(t_1)**2 + 2.0*sin(h)**4*cos(t_0 - t_1) + 0.5*sin(h)**4 - 2.0*sin(h)**2*cos(t_0 - t_1) - 2.0*sin(h)**2 + 3.0


In [13]:
R0=np.eye(3);
listR=[R0];
t=np.linspace(0,2*np.pi,11);
for k in t:
  Omega_k=[np.sin(k),np.cos(k),0];
  theta_k=(2*np.pi/11)*linalg.norm(Omega_k);
  n_k=Omega_k/linalg.norm(Omega_k);
  q_k=mr.qFromAxisAngles(theta_k,n_k)
  R0=R0 @ mr.RfromQuaternions(q_k);
  listR+=[R0]

In [ ]:
n=11
fig=go.Figure()
fig=mr.addOrthNormFrame(fig,[0,0,0],np.eye(3),[[-1.5,1.5],[-1.5,1.5],[-1.5,1.5]],'blue')
fig=mr.addOrthNormFrame(fig,[0,0,0],listR[n],[[-1.5,1.5],[-1.5,1.5],[-1.5,1.5]],'red')
fig.show()

In [17]:
XX0=mr.cube_vertices([2,1,0.5])
XX=[];
for R in listR:
  XX+=[[R @ XX0]]
fig=mr.animated_cube_flat_shading(XX, 'Test')